<a href="https://colab.research.google.com/github/nuvard/Where-to-eat/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import time


def get_near_info_strict(dr, address, fp):
#     Осуществляет загрузку страницы
#     dr - selenium chromedriver
#     address - адресс поиска (часть поискового запроса)
#     fp - handle -файла для записи в лог
#     Возвращает dict с описанием

    point_info = []
    
    url = 'https://yandex.ru/maps/?mode=search&text='
    url = url + address.replace(' ', '%20')

    try:
        dr.get(url)
    except Exception as e: 
        #print(e)
        fp.write(e)
        return
    time.sleep(.25)       # ждём загрузку

    point_info = find_onpage(dr,fp)
    return point_info


def find_onpage(dr, fp):
#     Ищет инфу на страницу
#     dr - selenium chromedriver
#     fp - handle -файла для записи в лог
#     Возвращает dict с описанием

    point_info = []
    
    try:
        element = dr.find_element_by_xpath("//div[@class='card-snippet-view__content']")
        element.click()
        time.sleep(.21)
    except:
        #print('Нет заголовка-описания точки')
        fp.write('Нет заголовка-описания точки')

        # Пытаемся понять что происходит
        try:
            # Может быть нам предлагают варианты выбрать? Берём первый попавшийся
            element = dr.find_element_by_xpath("//li[@class='search-snippet-view']")
            element.click()
            time.sleep(.21)

            # Что в этом доме
#            inplace = []
#            try:
#                element = dr.find_elements_by_xpath("//div[@class='business-card-title-view__categories']")
#                for i in range(len(element)):
#                    el = element[i]
#                    inplace.append(el.text.upper())
#            except:
#                print('Нет поля "В этом доме"')

            # Нам предлагают исправление? берём первый
            #try:
            #    element_1 = dr.find_element_by_xpath("//span[@class='search-command-view__show-results-button']")
            #    element_1.click()
            #    time.sleep(1)
            #except:
            #    print('Нет поля "Показать результаты по запросу"')
            return find_onpage(dr,fp)
            
        except:
            # print('Нет списка поиска')
            fp.write('Нет списка поиска')
            return
        
    descr = element.text.split('\n')        # описание того, что нашли с координатами
    
    element = dr.find_element_by_xpath("//div[@class='card-actions-view__item']")
    element.click()
    element = dr.find_element_by_xpath("//div[@class='card-share-view__text']")
    #with open('temp.png', 'wb') as fppng:
    #    fppng.write(element.screenshot_as_png)
    
    pos = [ float(element.text.split(',')[0]), float(element.text.split(',')[1]) ]
    
    
    stop_dist = -1
    stops = -1
    try:
        element = dr.find_element_by_xpath("//div[@class='masstransit-stops-view']")
        stop_dist = element.text.split('\n')
        if len(stop_dist)<3:     # одна остановка
            stops = 1    
        else:                    # много остановок
            stops = element.text.split('\n')[2]
        stop_dist = stop_dist[1]
        if stop_dist.find(",")>-1:     # километры ---> метры
            stop_dist = int(stop_dist.split(' ')[0].replace(',',''))*100
        else:
            stop_dist = int(stop_dist.split(' ')[0])
    except:
        #print('Нет поля "Остановки"')
        #print(fp.name)
        fp.write('Нет поля "Остановки"')
#        return
    
    # Что в этом доме
    inplace = []
    try:
        element = dr.find_elements_by_xpath("//div[@class='search-snippet-categories-view__category']")
        for i in range(len(element)):
            el = element[i]
            inplace.append(el.text.upper())
    except:
#        print('Нет поля "В этом доме"')
         fp.write('Нет поля "В этом доме"')
    
    point_info.append(descr[0])
    point_info.append(descr[1])
    #point_info.append(descr[2].split(','))
    point_info.append([pos[0], pos[1]])
    point_info.append(stops)
    point_info.append(stop_dist)
    point_info.append(inplace)

#     [type][numb][categories][category]
#     point_near[0][3][2][1]
    return point_info



'''
Загатовочка на будущее
    # Каталог "Искать рядом:"
    near = []
    
#     temp = None
#     try:
#         temp = dr.find_element_by_xpath("//div[@class='additional-results-view__hint']")
#     except Exception as e: 
#         print(e)
    catalog = dr.find_elements_by_xpath("//div[@class='catalog-small-item-view__icon']")
    length = len(catalog)    
#     print('if = ', length, catalog)
    if length > 0:
#         length = 8
        for i in range(length):
            catalog = dr.find_elements_by_xpath("//div[@class='catalog-small-item-view__icon']")
#             print('for = ',length, catalog)
            catalog[i].click()  # переход в каталог
            time.sleep(1)       # ждём загрузку
            # Достаём описание объектов
            element = dr.find_elements_by_xpath("//li[@class='search-snippet-view']")
            near_info = []
            for i in range(len(element)):
                el = element[i]
                # название
                el_title = el.find_element_by_xpath(".//div//div[starts-with(@class,'search-business-snippet-view__title')]")
        #         print(el_title.text )
                # категории
                cats = []
                el_type = el.find_elements_by_xpath(".//div//div//div/div[@class='search-snippet-categories-view']/div")
                for i in range(len(el_type)):
                    cats.append(el_type[i].text )
                # адресс
                el_address = el.find_element_by_xpath(".//div//div//div/div[@class='search-business-snippet-view__address']")
        #         print('address:', el_address.text)
                el_info = []
                el_info.append(el_title.text)
                el_info.append(el_address.text)
                el_info.append(cats)
                near_info.append(el_info)
            near.append(near_info)

            sleep_to = random.random()*2+1
            time.sleep(sleep_to)      # ждём загрузку

            # Возвращаемся на страницу с объектом
            element = dr.find_element_by_xpath("//div[@class='sidebar-panel-header-back-view__label']")
            element.click()
            time.sleep(1)       # ждём загрузку
    else:
        print('Не определил точно место')
        
    point_info.append(near)

#     [type][numb][categories][category]
#     point_near[0][3][2][1]
    return point_info
'''

# Web-element screenshot.
# with open('temp.png', 'wb') as outfile:  
#     outfile.write(el_type[1].screenshot_as_png)

# """
# Чтобы найти другие остановки, а не только ближайшую
# //div[@class='card-dropdown-view__content']
# stop_dist = element.text.split('\n')
# if len(stop_dist)<2:
#     stops = []
#     stops.append( stop_dist[0])
# else:
# #     card-dropdown-view__content
#     stop_dist = stop_dist[2]
#     stops = []
#     for i in range(4,int(stop_dist)*2+2+1,2):  # сколько рядом разных остановок и расстояние до них
#         stops.append( element.text.split('\n')[i].split(' ')[0])
# """
    


'\nЗагатовочка на будущее\n    # Каталог "Искать рядом:"\n    near = []\n    \n#     temp = None\n#     try:\n#         temp = dr.find_element_by_xpath("//div[@class=\'additional-results-view__hint\']")\n#     except Exception as e: \n#         print(e)\n    catalog = dr.find_elements_by_xpath("//div[@class=\'catalog-small-item-view__icon\']")\n    length = len(catalog)    \n#     print(\'if = \', length, catalog)\n    if length > 0:\n#         length = 8\n        for i in range(length):\n            catalog = dr.find_elements_by_xpath("//div[@class=\'catalog-small-item-view__icon\']")\n#             print(\'for = \',length, catalog)\n            catalog[i].click()  # переход в каталог\n            time.sleep(1)       # ждём загрузку\n            # Достаём описание объектов\n            element = dr.find_elements_by_xpath("//li[@class=\'search-snippet-view\']")\n            near_info = []\n            for i in range(len(element)):\n                el = element[i]\n                # наз

In [6]:
!pip install selenium
from selenium import webdriver

     |████████████████████████████████| 911kB 4.9MB/s 


In [20]:
!git clone https://github.com/nuvard/Where-to-eat.git

path = '/content/Where-to-eat/'

Cloning into 'Where-to-eat'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 28 (delta 7), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [0]:
path = '/content/Where-to-eat/'

In [22]:
driver = webdriver.Chrome(path + "chromedriver")

WebDriverException: ignored

In [0]:
import pandas as pd

rests = pd.read_csv(path+'data/'+'restoclub_new.csv', index_col=0)

In [10]:
rests.tail()

,name,link,rating,about,address,смешанная,европейская,check,rating_inside,живая музыка,шоу-программа,русская,детское меню,завтрак,бизнес-ланч,при отеле,летняя терраса,настольные игры,десерты,рыба и морепродукты,детская комната,ресторан у воды,парковка,стейки,спортивные трансляции,кулинарные мастер-классы,камин,коктейли,dj,стриптиз,кабинки,вегетарианская,за городом,панорамный вид,заказ кейтеринга,крафтовое пиво,доставка,авторская,торты на заказ,кальян,...,американская,мексиканская,латиноамериканская,блюда из дичи,еврейская,шаверма,корейская,можно с животными,сербская,гастрономические сеты,бельгийская,греческая,пышки,скандинавская,испанская,перуанская,шотландская,узбекская,турецкая,португальская,тайская,вьетнамская,английская,ирландская,фьюжн,абхазская,норвежская,африканская,австрийская,татарская,бурятская,ливанская,здесь живут котики,арабская,кубинская,финская,латышская,lat,lng,in_this_house
4307,Sennaya4,/spb/place/sennaya4,0.0,0,пл. Сенная 4,0.0,0.0,650 (без напитков)\r\r\r\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
4308,Гриль Factory,/spb/place/gril-factory-2,0.0,0,ул. Адмирала Трибуца 6к1,1.0,1.0,600 (без напитков)\r\r\r\n,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
4309,Пхали хинкали,/spb/place/phali-hinkali-6,0.0,0,Невский проспект 32-34,1.0,1.0,850 (без напитков)\r\r\r\n,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
4310,Чифальна,/spb/place/chifalna,0.0,0,Лиговский пр. 74,0.0,0.0,400 (без напитков)\r\r\r\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
4311,Probirka Bar,/spb/place/probirka-bar,0.0,0,ул. Ломоносова 2,1.0,1.0,1000 (без напитков)\r\r\r\n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0


In [0]:
rests.columns

Index(['name', 'link', 'rating', 'about', 'address', 'смешанная',
       'европейская', 'check', 'rating_inside', 'живая музыка',
       ...
       'бурятская', 'ливанская', 'здесь живут котики', 'арабская', 'кубинская',
       'финская', 'латышская', 'lat', 'lng', 'in_this_house'],
      dtype='object', length=116)

In [0]:
rests.columns[[0,4]]

Index(['name', 'address'], dtype='object')

In [0]:
#rests['lat']=0

In [0]:
#rests['lng']=0

In [0]:
rests=rests[rests.columns[[0,4]]]

In [0]:
search = 'ул. Большая Конюшенная 2'
url = 'https://yandex.ru/maps/2/saint-petersburg/?ll=30.324330%2C59.939555&mode=search&sll=30.319923%2C59.927287&source=wizgeo&sspn=0.754373%2C0.214831&' \
    + 'text='+ search.replace(' ', '%20') 

In [0]:
#driver.get(url)

In [0]:
#content = driver.find_element_by_css_selector('div.clipboard__action-wrapper')

In [0]:
#content.text.split(' ')

['59.939555,', '30.324330']

import numpy as np
rests['lat'] = np.zeros(len(rests))
rests['lng'] = np.zeros(len(rests))

In [0]:
temp_rests = pd.DataFrame(columns = ['name','address','lat', 'lng', 'review_count', 'reviews',\
                                     'rating_yandex', 'features' , 'link'])

In [0]:
temp_rests = pd.read_csv(path+'data/yandex_rests_preprocessed.csv')

In [14]:
temp_rests.tail()

,address,lat,link,lng,name,rating_yandex,review_count,reviews,rating_yandex_norm,review_count_norm,popularity,kitchen,DJ,барная стойка,бесплатная парковка,винная карта,живая музыка,закрытие под банкет,круглосуточная кухня,меню на английском,местоположение у воды,настольные игры,своя пекарня,устричный бар,24_hour,DJ,Restopub Finnegan's приглашает на бизнес ланч,discount_10,discount_15,discount_20,discount_50,discount_7,Завтраки в ресторане МА,Счастливые часы,барная стойка,бесплатная парковка,бранчи,винная карта,детская анимация,живая музыка,...,кухня_домашняя,кухня_еврейская,кухня_европейская,кухня_индийская,кухня_индонезийская,кухня_ирландская,кухня_испанская,кухня_итальянская,кухня_кавказская,кухня_китайская,кухня_континентальная,кухня_корейская,кухня_латиноамериканская,кухня_международная,кухня_мексиканская,кухня_морская,кухня_мясная,кухня_национальная,кухня_немецкая,кухня_неткухня_специализации,кухня_паназиатская,кухня_русская,кухня_рыбная,кухня_сербская,кухня_скандинавская,кухня_смешанная,кухня_средиземноморская,кухня_тайская,кухня_узбекская,кухня_украинская,кухня_французская,кухня_фьюжн,кухня_чешская,кухня_шашлык,кухня_японская,меню на английском,местоположение у воды,настольные игры,своя пекарня,танцпол
1675,пр. Московский 139/1,59.879864,https://yandex.ru/maps/org/gent/112945565451/,30.318527,Гент,4.4,170.0,"Так, начнём! Моя продюсерская судьба кидала ме...",0.842105,0.069237,0.074408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1676,пр. Каменноостровский 22,59.960791,https://yandex.ru/maps/org/coffee_room/1155017...,30.315626,Coffee Room,4.2,132.0,Кафе посетили по старой памяти. Ранее всегда ц...,0.789474,0.053483,0.053885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1677,ул. Савушкина 21 лит. А,59.986079,https://yandex.ru/maps/org/yunost/158639523050/,30.285828,Юность,4.7,629.0,"Понтов много, ценник высокий, в субботу днем п...",0.921053,0.259536,0.305068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1678,пер. Перекупной 3,59.927617,https://yandex.ru/maps/org/bombay/86669928054/,30.376774,BombaY,4.3,183.0,"На любителя.Зловонные приправы,не опытные пова...",0.815789,0.074627,0.077694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,В.О. пр. Большой 19,59.940078,https://yandex.ru/maps/org/tan_zhen/1042818239/,30.283987,Тан Жен,4.4,431.0,"Здоровский ресторан, но понимаете, что размер ...",0.842105,0.177446,0.190699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
import time

In [0]:
types = ['ресторан','бар','паб','кафе' ,'кофейня','столовая','бистро', \
         'пиццерия', 'пекарня', 'быстрое питание', 'спортбар'\
        ]


In [0]:
errors = pd.DataFrame(columns = ['index', 'address', 'num_of_card'])

In [0]:
errors = pd.read_csv(path+'data/errors.csv', index_col=0)

In [0]:
errors

,index,address,num_of_card


In [0]:
print(rests.iloc[-1,-1])


0


In [0]:
rests['in_this_house'] = rests['in_this_house'].replace(0,'')

In [0]:
rests = rests.fillna(0)

In [0]:
temp_rests.iloc[-1, 0]

'ул. Большая Конюшенная\xa01'

In [0]:
rows_done = 0

In [0]:
bad_links =  [' ', '', 'Маршрут'] 

In [0]:
rests['address'] = rests['address'].replace('\xa0', ' ' )

In [0]:

temp_rests.to_csv('yandex_rests.csv', encoding='utf-8', index=True)
errors.to_csv('errors.csv', encoding='utf-8', index=True)
rests.to_csv('restoclub_new.csv', index=True)

In [0]:
bad_features = ['ул.', '+7', 'пр.', 'ш.', 'Показать телефон', '.ru', ' м', 'Посмотреть всё меню', '?', \
               'Закрыто до','До открытия ', 'Посмотреть всё меню', 'просп', 'Соцсети', 'Подробнее', 'com','Нет информации', 'Все филиалы сети',\
               'наб', 'Добавить организацию', 'Показать парковки','смотреть', '.fi','.pro','.info', 'смотреть', 'пер.', 'км', 'фото'\
               'линия', '.su', 'Открыто до','₽', 'Где', 'Еще', 'подбор', 'площадь', 'Показать входы','!','Кухня:', 'Особенности заведения:' ]
def clean(features):
  good_features = []
  counter = 0;
  for feature in features:
      if(feature.find('Кухня:')!=-1):
          good_features.extend(feature.split(':')[1].replace(' ','кухня_').split(','))
          #features.pop(feature)
      if(feature.find('Особенности заведения:')!=-1):
          good_features.extend(feature.split(':')[1].split(', '))
      if feature.find('%')!=-1:
          feature = 'discount_'+str(abs(int(feature.split('%')[0].split(' ')[-1])))
      if feature.lower().find('круглосуточно')!=-1:
            feature = '24_hour'
             
      for i in bad_features:
          
          if feature.find(i)!=-1:
              counter+=1
              continue 
          
      if counter==0:
          good_features.append(feature)
  return list(set(good_features))

def find_kitchen(features):
    for feature in features:
        if(feature.find('Кухня:')!=-1):
            features.append(feature.split(':')[1].replace(' ','кухня_').split(','))
            features.pop(feature)
    return features
  
  
def find_features(features):
    for feature in features:
        if(feature.find('Особенности заведения:')!=-1):
            features.append(feature.split(':')[1].split(', '))
            features.pop(feature)
    return features


In [17]:
iterator = 0
flag = 0
counter = 0 
rests['address'] = rests['address'].replace('\xa0', ' ' )
for index, item in rests.iterrows():
    if(item['address']=='В.О. пр. Большой\xa019'):
        flag=1
        print('begin')
    if(flag==0):
        continue
    counter+=1
    iterator+=1
    search = item['address']
    url = 'https://yandex.ru/maps/2/saint-petersburg/?ll=30.324330%2C59.939555&mode=search&sll=30.319923%2C59.927287&source=wizgeo&sspn=0.754373%2C0.214831&' \
        + 'text='+ search.replace(' ', '%20') 
    try: 
        driver.get(url)
        time.sleep(.15)  
    except:
        time.sleep(60)
        driver.get(url)
        time.sleep(.15)  
    try:
        coords = driver.find_element_by_css_selector('.clipboard__action-wrapper').text.split(' ')
        rests.loc[index, ['lat', 'lng']] = coords
    except:
        new_error = dict()
        new_error['index']=[index]
        new_error['address']=[search]
        new_error['num_of_card']=[k]
        errors.append(pd.DataFrame(new_error))
    content = driver.find_elements_by_css_selector('.search-snippet-categories-view__category')
    categories = []
    categories = [cat.text.lower() for cat in content]
    #print('cats',','.join(categories))
    #print('rest' , rests.loc[index, 'in_this_house'])
    rests.loc[index, 'in_this_house']+=','.join(categories)
    #print(categories)
    cards = driver.find_elements_by_css_selector('.search-snippet-view')
    #print(len(cards))
    content = driver.find_elements_by_xpath("//div[@class = 'search-business-snippet-view']")
    #print(len(content))
    links = driver.find_elements_by_css_selector('.link-wrapper')
    names = [link.text for link in links]
    #print(names)
    good_i = [i for i in range(len(links)) if all([names[i]!=t for t in bad_links])]
    links = [links[i] for i in good_i]
    names = [names[i] for i in good_i]
    #print(names)
    for k in range(len(cards)):
        driver.get(url) 
        content = driver.find_elements_by_xpath("//div[@class = 'search-business-snippet-view']")
        try:
            current_cats = ' '.join([cat.text for cat in content[k].find_elements_by_css_selector('.search-snippet-categories-view__category')]) 
            #print(current_cats)
            #links = driver.find_elements_by_css_selector('.link-wrapper')
            temp_index = len(temp_rests)
            if any([current_cats.lower().find(t)!=-1 for t in types]):
                links = (driver.find_elements_by_css_selector('.link-wrapper'))
                links = [links[i] for i in good_i]
                con = links[k]
                new_rest = dict()
                new_rest['link'] = con.get_attribute("href")
                #name = con.text.split('\n')[0]
                con.click()
                features = [f.text for f in driver.find_elements_by_css_selector('.card-feature-view__main')] 
                features = clean(features)
                rating = driver.find_element_by_css_selector('.business-rating-badge-view__rating-text').text
                reviews = driver.find_element_by_css_selector('.business-rating-amount-view').text
                reviews_texts = [r.text for r in driver.find_elements_by_css_selector('.text-ellipsis-view')]                  

                new_rest['name']=names[k]
                #print(new_rest['name'])
                new_rest['address'] = search
                new_rest['review_count'] = reviews.split(' ')[0]
                new_rest['rating_yandex'] = rating.split(' ')[0]
                #for i in range(reviews_texts):
                #    new_rests['review_'+str(i)] = reviews_texts
                new_rest['reviews'] = ("|").join(reviews_texts)
                #for feature in features:
                #    new_rest[feature]=1
                new_rest['features'] = ("|").join(features)
                for feature in features:
                  new_rest[feature]=1
                new_rest['lat'] = coords[0]
                new_rest['lng'] = coords[1]
                temp_rests = pd.concat([temp_rests, pd.DataFrame(new_rest, index = [temp_index])])
        except:
                new_error = dict()
                new_error['index']=[index]
                new_error['address']=[search]
                new_error['num_of_card']=[k]
                errors.append(pd.DataFrame(new_error))
                time.sleep(.15)
    if (counter % 20 == 0):
        temp_rests.to_csv('yandex_rests.csv', encoding='utf-8', index=True)
        errors.to_csv('errors.csv', encoding='utf-8', index=True)
        rests.to_csv('restoclub_new.csv', index=True)


            
temp_rests.to_csv('yandex_rests.csv', encoding='utf-8', index=True)
errors.to_csv('errors.csv', encoding='utf-8', index=True)
rests.to_csv('restoclub_new.csv', index=True)

begin


NameError: ignored

In [0]:
temp_rests.to_csv('yandex_rests.csv', encoding='utf-8', index=True)
errors.to_csv('errors.csv', encoding='utf-8', index=True)
rests.to_csv('restoclub_new.csv', index=True)


In [0]:
rests.tail()

,name,link,rating,about,address,смешанная,европейская,check,rating_inside,живая музыка,...,бурятская,ливанская,здесь живут котики,арабская,кубинская,финская,латышская,lat,lng,in_this_house
4307,Sennaya4,/spb/place/sennaya4,0.0,0,пл. Сенная 4,0.0,0.0,650 (без напитков)\r\r\n,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN
4308,Гриль Factory,/spb/place/gril-factory-2,0.0,0,ул. Адмирала Трибуца 6к1,1.0,1.0,600 (без напитков)\r\r\n,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN
4309,Пхали хинкали,/spb/place/phali-hinkali-6,0.0,0,Невский проспект 32-34,1.0,1.0,850 (без напитков)\r\r\n,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN
4310,Чифальна,/spb/place/chifalna,0.0,0,Лиговский пр. 74,0.0,0.0,400 (без напитков)\r\r\n,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN
4311,Probirka Bar,/spb/place/probirka-bar,0.0,0,ул. Ломоносова 2,1.0,1.0,1000 (без напитков)\r\r\n,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN


In [0]:
driver.get(url)

In [0]:
driver.get(url)
content = driver.find_elements_by_css_selector('div.search-snippet-categories-view__category')
for k in range(len(content)):
    i=content[k]
    print(i.text.lower())
    k+=1
    if any([i.text.lower().find(t)!=-1 for t in types]):
        con = i.find_element_by_xpath('../../..')
        print(str(i)+  ' name ' + con.text)
        con.click()
        rating = driver.find_element_by_css_selector('span.business-rating-badge-view__rating-text').text
        reviews = driver.find_element_by_css_selector('span.business-rating-amount-view').text
        reviews_texts = [i.text for i in driver.find_elements_by_css_selector('span.text-ellipsis-view')]
        print(reviews_texts)
        print(rating, reviews)
        features = [i.text for i in driver.find_elements_by_css_selector('div.card-feature-view__main')] 
        print(features)
        driver.back()
        content = driver.find_elements_by_css_selector('div.search-snippet-categories-view__category')


банк
обмен валюты
пиццерия
ресторан
<selenium.webdriver.remote.webelement.WebElement (session="aa2801feb164e7c1c885fb8a97279837", element="10e7f2ed-0254-4e2e-9f8e-ec9be2d8909c")> name Пицца 22 сантиметра
Пиццерияресторан
['Цены завышены, обслуживание отличается отсутствием элементарных начал вежливости и внимания. Изначально как то не задалось с размещением, официант... ещё', 'уникальное место с очень вкусной и необычной пиццей', 'Пицца отличная, готовят быстро, обслуживание отвратительное-не понравилась работа официанток, или им пофигу или просто устали к концу дня... ещё', 'Оригинальные рецепты, очень вкусно. Рекомендую вариант пиццы с тамбовским окороком. Бронируйте заранее, очень большой поток посетителей. Цены очень... ещё', 'Возможно это фишка пиццерии, но горела корочка очень невкусная(((( и начинка так себе. Больше не пойду, это точно']
4.4 292 оценки
['Посмотреть всё меню', 'Открыто до 23:00', '+7 (812) 200-93-85\nПоказать телефон', 'Большая Конюшенная ул., 2', 'pizza22cm.ru',

In [0]:
input_el.find_element_by_xpath('..')